# LUME-services

### Configuration:

Set the following environment variables:

```
export LUME_MODEL_DB__HOST=127.0.0.1
export LUME_MODEL_DB__USER=root
export LUME_MODEL_DB__PASSWORD=test
export LUME_MODEL_DB__PORT=3306
export LUME_MODEL_DB__DATABASE=model_db
export LUME_MODEL_DB__CONNECTION__POOL_SIZE=1
export LUME_PREFECT__SERVER__TAG=core-1.2.4
export LUME_PREFECT__SERVER__HOST_PORT=4200
export LUME_PREFECT__SERVER__HOST=http://localhost
export LUME_PREFECT__AGENT__HOST_PORT=5000
export LUME_PREFECT__AGENT__HOST=http://localhost
export LUME_RESULTS_DB__HOST=127.0.0.1
export LUME_RESULTS_DB__PORT=27017
export LUME_RESULTS_DB__USERNAME=root
export LUME_RESULTS_DB__PASSWORD=password
export LUME_RESULTS_DB__DATABASE=test
export LUME_PREFECT__BACKEND=server 
export LUME_BACKEND=docker
export LUME_ENVIRONMENT__LOCAL_PIP_REPOSITORY=...
export LUME_ENVIRONMENT__LOCAL_CONDA_CHANNEL_DIRECTORY=...
```

Start the docker services using:
```
lume-services docker start-services
```

In [1]:
from lume_services.services.models.db import ModelDBConfig
from lume_services.services.results.mongodb import MongodbResultsDBConfig
from lume_services.services.scheduling.backends.server import (
    PrefectAgentConfig,
    PrefectConfig,
    PrefectServerConfig,
)

from lume_services.services.files.filesystems import (
    LocalFilesystem,
    MountedFilesystem,
)
from lume_services.config import LUMEServicesSettings
import logging
logging.basicConfig(level="DEBUG")

model_db_config = ModelDBConfig(
        host="127.0.0.1",
        port="3306",
        user="root",
        password="test",
        database="model_db",
)

results_db_config = MongodbResultsDBConfig(
    port="27017",
    host="127.0.0.1",
    username="root",
    database="test",
    password="password",
)

prefect_config = PrefectConfig(
    server=PrefectServerConfig(
        host="http://localhost", host_port="4200", tag="core-1.2.4"
    ),
    agent=PrefectAgentConfig(host="http://localhost", host_port="5000"),
    backend="server",
)


settings = LUMEServicesSettings(
    model_db=model_db_config,
    results_db=results_db_config,
    prefect=prefect_config,
    backend="docker",
    mounted_filesystem=None,
)

## Use LUME-services config to create injected services

In [2]:
from lume_services import config
config.configure(settings)

# initialize singletons
model_db_service = config.context.model_db_service()
results_db_service = config.context.results_db_service()
scheduling_service = config.context.scheduling_service()

INFO:lume_services.config:Configuring LUME-services environment...
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
INFO:lume_services.config:Environment configured.
DEBUG:lume_services.config:Environment configured using {'model_db': {'host': '127.0.0.1', 'port': 3306, 'user': 'root', 'database': 'model_db', 'connection': {'pool_size': None, 'pool_pre_ping': True}, 'dialect_str': 'mysql+pymysql'}, 'results_db': {'username': 'root', 'host': '127.0.0.1', 'port': 27017, 'authMechanism': 'DEFAULT'}, 'prefect': {'server': {'tag': 'core-1.2.4', 'host': 'http://localhost', 'host_port': '4200', 'host_ip': '127.0.0.1'}, 'ui': {'host': 'http://localhost', 'host_port': '8080', 'host_ip': '127.0.0.1', 'apollo_url': 'http://localhost:4200/graphql'}, 'telemetry': {'enabled': True}, 'agent': {'host': 'http://localhost', 'host_port': '5000'}, 'home_dir': '~/.p

## if you're running this many time, creation will fail because of uniqueness... You can reset since this is a dev server

In [3]:
model_db_service._reset()

## Store example model, deployment, flow, and project

In [4]:
from lume_services.models import Model

model = Model.create_model(
    author = "Jackie Garrahan",
    laboratory = "slac",
    facility = "lcls",
    beampath = "cu_hxr",
    description = "test_model"
)

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.


In [5]:
# create a project
project_name = model_db_service.store_project(
    project_name="test", description="my_description"
)

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO project (project_name, description) VALUES (:project_name, :description)
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO project (project_name, description) VALUES (:project_name, :description)


In [6]:
scheduling_service.create_project("test")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 68
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 74


## Let's use the source of the model we created in the demo

In [ ]:
source_path = "https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.7/my_model-0.0.7.tar.gz"
# populates local channel
model.store_deployment(source_path, project_name="test")

INFO:lume_services.environment.solver:https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.7/my_model-0.0.7.tar.gz saved to /tmp/lume-services/my_model-0.0.7.tar.gz
INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO deployment (version, asset_dir, source, sha256, image, is_live, model_id) VALUES (:version, :asset_dir, :source, :sha256, :image, :is_live, :model_id)
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO deployment (version, asset_dir, source, sha256, image, is_live, model_id) VALUES (:version, :asset_dir, :source, :sha256, :image, :is_live, :model_id)
INFO:lume_services.environment.solver:https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.7/my_model-0.0.7.tar.gz saved to /tmp/lume-services/my_model-0.0.7.tar.gz
INFO:lume_services.environment.solver:Using conda ch

DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/graphviz-5.0.1-ha8464fc_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/pure_eval-0.2.2-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/lxml-4.9.1-py39h701faf5_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/gts-0.7.6-hccb3bdf_2.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/platformdirs-2.5.2-pyhd8ed1ab_1.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/importlib-metadata-4.11.4-py39h6e9494a_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/toml-0.10.2-pyhd8ed1ab_0.

DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/mypy_extensions-0.4.3-py39h6e9494a_5.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/identify-2.5.3-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/cached_property-1.5.2-pyha770c72_1.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/locket-1.0.0-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/urllib3-1.26.11-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/xorg-libxdmcp-1.1.3-h35c211d_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/pandas-1.4.4-p

DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/ghp-import-2.1.0-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/cctools-973.0.1-h351d84c_10.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/cffi-1.15.1-py39hae9ecf2_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/sniffio-1.2.0-py39h6e9494a_3.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/asttokens-2.0.8-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/markdown-it-py-2.1.0-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/python-libarchive-c-4.0

DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/readline-8.1.2-h3899abd_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/mkdocstrings-0.19.0-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/charset-normalizer-2.1.1-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/libllvm13-13.0.1-h64f94b2_2.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/defusedxml-0.7.1-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/pyopenssl-22.0.0-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jgarra/miniconda3/envs/lume-services-dev/conda-meta/pip-22.2.2-pyhd

...

INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/linux-64/libarchive-3.5.2-hb890918_3.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/linux-64/sqlite-3.39.2-h4ff8645_1.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/linux-64/brotli-1.0.9-h166bdaf_7.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/linux-64/libcurl-7.83.1-h7bff187_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/linux-64/openjpeg-2.5.0-h7d73246_1.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/linux-64/lcms2-2.12-hddcbb42_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/linux-64/hdf5-1.12.2-nompi

INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/noarch/partd-1.3.0-pyhd8ed1ab_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/noarch/importlib_resources-5.9.0-pyhd8ed1ab_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/noarch/packaging-21.3-pyhd8ed1ab_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/noarch/croniter-0.3.36-pyhd8ed1ab_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/noarch/marshmallow-3.17.1-pyhd8ed1ab_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/noarch/marshmallow-oneofschema-3.0.1-pyhd8ed1ab_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-c

INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/linux-64/conda-build-3.22.0-py39hf3d152e_2.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/linux-64/mamba-0.25.0-py39hfa8f2c8_2.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jgarra/sandbox/lume-services/dev/local-channel/noarch/prefect-1.3.0-pyhd8ed1ab_0.tar.bz2
  Running command python setup.py egg_info
  running egg_info
  creating /private/var/folders/5k/4l30k4v90kq1rlqkglct0j4cp8kcc2/T/pip-pip-egg-info-p4keu8ke/lume_services.egg-info
  writing /private/var/folders/5k/4l30k4v90kq1rlqkglct0j4cp8kcc2/T/pip-pip-egg-info-p4keu8ke/lume_services.egg-info/PKG-INFO
  writing dependency_links to /private/var/folders/5k/4l30k4v90kq1rlqkglct0j4cp8kcc2/T/pip-pip-egg-info-p4keu8ke/lume_services.egg-info/dependency_links.txt
  writing entry points to /private/var/folders/5k/4l30k4v90kq1rlqkglct0j4cp8kcc2/T/pip-pip-e

conda-forge/osx-64                                          Using cache
conda-forge/noarch                                          Using cache


DEBUG:lume_services.environment.solver:Channels: ['conda-forge']
DEBUG:lume_services.environment.solver:Platform: osx-64
DEBUG:lume_services.environment.solver:Output_folder: /Users/jgarra/sandbox/lume-services/dev/local-channel


In [ ]:
model.run(parameters={
                        "input1": "hey", 
                        "input2": "jackie", 
                        "filename": f"/Users/jgarra/sandbox/lume-services/test_file.txt", 
                        "filesystem_identifier":"local"}
         )

In [ ]:
model.results()